In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [20]:
# Read the growth rate data
yearly_growth = pd.read_csv('../data/landing/yearly_growth_rate_of_median_rent_by_suburb.csv')
yearly_growth.head()

,suburb,year,growth_rate
0,Albert Park-Middle Park-West St Kilda,2003,-1.666667
1,Armadale,2003,0.858369
2,Carlton North,2003,0.000000
3,Carlton-Parkville,2003,3.703704
4,CBD-St Kilda Rd,2003,-3.125000


In [27]:

# Build a model to predict the yearly growth rate of the median rent for a given suburb and year

le = LabelEncoder()
X = yearly_growth[['suburb', 'year']]
X['suburb'] = le.fit_transform(X['suburb'])
y = yearly_growth['growth_rate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30034)

# Fit a random forest regressor assuming the predictors are independent
RFR = RandomForestRegressor(n_estimators=100, random_state=30034)
RFR.fit(X_train, y_train)
y_pred = RFR.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse}')

# Predict the growth rate for each suburb in 2024
suburbs = le.fit_transform(yearly_growth['suburb'].unique())
years = [2024] * len(suburbs)
X_pred = pd.DataFrame({'suburb': suburbs, 'year': years})
y_pred = RFR.predict(X_pred)

# The top 10 suburbs with the highest predicted growth rate in 2024
suburbs = le.inverse_transform(suburbs)
growth_rate = pd.DataFrame({'suburb': suburbs, 'growth_rate': y_pred})
growth_rate = growth_rate.sort_values('growth_rate', ascending=False)
growth_rate.head(10)


/tmp/ipykernel_2759/1766429523.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['suburb'] = le.fit_transform(X['suburb'])


Mean squared error: 16.473171092781257


,suburb,growth_rate
28,Canterbury-Surrey Hills-Mont Albert,31.710033
19,Southbank,28.986747
2,Carlton North,28.173828
27,Camberwell-Glen Iris,28.125757
4,CBD-St Kilda Rd,27.026678
3,Carlton-Parkville,25.120764
43,Carnegie,24.778679
6,Docklands,22.382418
18,South Yarra,20.411094
44,Caulfield,20.045454
